# Orphaned Blocks Analyzer
Chart the distribution of orphaned blocks, show top winners and losers. 

In [1]:
import glob
import json
from pandas import DataFrame
from pandas import json_normalize
import pandas
import requests 

# Load blocks from disk into dataframe 
def load_blocks_from_disk(path_to_blocks="./archive-blocks/"):
    block_files = glob.glob(path_to_blocks + "*.json")
    blocks = []
    for file in block_files: 
        with open(file) as fp:
            blocks.append(json.load(fp))
    return blocks


blocks_query = '''
query BlocksQuery {
  blocks(limit: 4000) {
    protocolState {
      consensusState {
        slot
        blockHeight
        blockchainLength
      }
    }
    canonical
    creator
    stateHash
    receivedTime
    dateTime
  }
}
'''
def load_blocks_from_block_explorer(url="https://graphql.minaexplorer.com/", limit=100):
    r = requests.post(url, json={'query': blocks_query})
    payload = json.loads(r.text)
    
    blocks = payload["data"]["blocks"]
    
    cleaned = []
    for block in blocks:
        cleaned.append({
            "slot": block["protocolState"]["consensusState"]["slot"],
            "blockHeight": block["protocolState"]["consensusState"]["blockHeight"],
            "canonical": block["canonical"],
            "creator": block["creator"],
            "stateHash": block["stateHash"],
            "receivedTime": block["receivedTime"],
            "dateTime": block["dateTime"],
        })
    return cleaned
        
blocks = load_blocks_from_block_explorer()
print(len(blocks))
df = DataFrame(blocks)
display(df)

2406


,slot,blockHeight,canonical,creator,stateHash,receivedTime,dateTime
0,0,1,True,B62qiy32p8kAKnny8ZFwoMhYpBppM1DWVCqAPBYNcXnsAH...,3NLxD2jjweB7an21Df2zoqDy5UBBui3m179A3XafunQZey...,2021-02-28T04:22:05Z,2021-02-28T04:22:05Z
1,873,2,True,B62qpge4uMq4Vv5Rvc8Gw9qSquUYd6xoW1pz7HQkMSHm6h...,3NKMTjTAud8yW1twPG38hoSmHN83Tv3wZ6jsJQ5HFhB1Uh...,2021-03-02T00:01:45.542Z,2021-03-02T00:01:05Z
2,930,3,True,B62qrCz3ehCqi8Pn8y3vWC9zYEB9RKsidauv15DeZxhzkx...,3NLN9CzC2bNQJLHGHP3pTYbLbgsKqxzFxMsDkwVRs4kTYx...,2021-03-02T02:52:31.828Z,2021-03-02T02:52:05Z
3,943,4,True,B62qmWbgvGV1MwxS6rJEG3BSbT2T8b5DP2Theb2CaBduBX...,3NKHMNNMKm9Vj4YG1CKmsWiJju3ti2bNLbPYeQKaMrcNVm...,2021-03-02T03:31:43.164Z,2021-03-02T03:31:05Z
4,945,5,True,B62qrr1gkgYvVfRb9JjstCcjDTKYYXFM61dD7oqFaEjc5V...,3NLcscsV8p1WCT7kiz4ChnynK8STymQHFzphsiawEefXqx...,2021-03-02T03:37:55.771Z,2021-03-02T03:37:05Z
...,...,...,...,...,...,...,...
2401,4288,1637,True,B62qjCuPisQjLW7YkB22BR9KieSmUZTyApftqxsAuB3U21...,3NKkLtAdzeJ5Nyzyat893U9SXr7SxLgRE2ehZFir7oNXMi...,2021-03-09T02:47:00.063Z,2021-03-09T02:46:05Z
2402,4296,1638,True,B62qoVopcNoQPFydweGWUBnJJbrokkebVDiWGmAzYoaLys...,3NKvJNXCjSpZPnCUb1KHH2gbpFD4XQfqNnrfm5hjuG1C6N...,2021-03-09T03:10:54.347Z,2021-03-09T03:10:05Z
2403,4299,1639,True,B62qjCuPisQjLW7YkB22BR9KieSmUZTyApftqxsAuB3U21...,3NKc9ErbfT5CotBzo7DsnNcoCLqxypKsXZCHU3yZZacmu1...,2021-03-09T03:19:58.571Z,2021-03-09T03:19:05Z
2404,4314,1640,True,B62qn7HrKKt5ia1dvGYHuvuFGLdwNSXUSAERQgvS2yZbZv...,3NKcH7UpfFcW3Z1ErsQVMHMgZ5YMNT8BcmESSua119KkNk...,2021-03-09T04:04:56.269Z,2021-03-09T04:04:05Z


In [6]:
vc = df["slot"].value_counts().reset_index(name="count")

pandas.set_option('display.max_rows', 500)
pandas.set_option('display.max_columns', 500)
pandas.set_option('display.width', 1000)

vc

,index,count
0,3769,6
1,4028,5
2,1675,5
3,2572,5
4,2595,5
...,...,...
1667,1722,1
1668,1720,1
1669,3759,1
1670,1708,1


In [15]:
fullSlots = df.slot.unique()
handicap = 1000
nFullSlots = len(df.slot.unique())
max_slot = 4324
# max_slot - (count of unique slots) = nEmptySlots
emptySlots = max_slot - nFullSlots - handicap
ratioEmpty = emptySlots/(max_slot-handicap)
print(f"Total Slots: {max_slot}")
print(f"Slot Handicap: {handicap}")
print(f"Filled Slots: {nFullSlots}")
print(f"Empty Slots: {emptySlots}")
print(f"Ratio Empty: {ratioEmpty}")

Total Slots: 4324
Slot Handicap: 1000
Filled Slots: 1672
Empty Slots: 1652
Ratio Empty: 0.4969915764139591


In [10]:
import plotly.express as px
fig = px.bar(vc, x="index", y="count")
fig.show()